In [12]:
import sqlite3
from pathlib import Path
import pandas as pd

RAW_CSV = Path(r"C:\Users\maitr\OneDrive\Desktop\DataScience\Project\data\medicaid_data.csv")
DB_DIR = Path(r"C:\Users\maitr\OneDrive\Desktop\DataScience\Project\database")
DB_PATH = DB_DIR / "medicaid_drugs.db"


print("CSV exists:", RAW_CSV.exists())
print("DB will be created at:", DB_PATH)


CSV exists: True
DB will be created at: C:\Users\maitr\OneDrive\Desktop\DataScience\Project\database\medicaid_drugs.db


In [13]:
# Cleaning Headers

df = pd.read_csv(RAW_CSV, low_memory=False)

df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

df.head()


,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,suppression_used,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,FFSU,AK,2143380,2,1433,80,2025,2,False,TRULICITY,216.0,107.0,102976.40,98630.87,4345.53
1,FFSU,AK,2143480,2,1434,80,2025,2,False,TRULICITY,218.0,109.0,104481.92,101806.64,2675.28
2,FFSU,AK,2143611,2,1436,11,2025,2,False,EMGALITY P,21.0,20.0,15227.25,15227.25,0.00
3,FFSU,AK,2144511,2,1445,11,2025,2,False,TALTZ AUTO,33.0,30.0,231532.28,231532.28,0.00
4,FFSU,AK,2144527,2,1445,27,2025,2,True,TALTZ AUTO,NaN,NaN,NaN,NaN,NaN


In [14]:
# Clean text columns
text_cols = [
    "utilization_type", "state", "ndc", "labeler_code",
    "product_code", "package_size", "suppression_used", "product_name"
]
for c in text_cols:
    df[c] = df[c].astype("string").str.strip()

# Convert numeric columns safely
num_cols = [
    "units_reimbursed",
    "number_of_prescriptions",
    "total_amount_reimbursed",
    "medicaid_amount_reimbursed",
    "non_medicaid_amount_reimbursed"
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Convert year and quarter to normal Python int
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["quarter"] = pd.to_numeric(df["quarter"], errors="coerce")

df = df.dropna(subset=["year", "quarter"])
df["year"] = df["year"].astype(int)
df["quarter"] = df["quarter"].astype(int)

df.info()


<class 'pandas.DataFrame'>
RangeIndex: 2637009 entries, 0 to 2637008
Data columns (total 15 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   utilization_type                string 
 1   state                           string 
 2   ndc                             string 
 3   labeler_code                    string 
 4   product_code                    string 
 5   package_size                    string 
 6   year                            int64  
 7   quarter                         int64  
 8   suppression_used                string 
 9   product_name                    string 
 10  units_reimbursed                float64
 11  number_of_prescriptions         float64
 12  total_amount_reimbursed         float64
 13  medicaid_amount_reimbursed      float64
 14  non_medicaid_amount_reimbursed  float64
dtypes: float64(5), int64(2), string(8)
memory usage: 301.8 MB


In [15]:


import sqlite3

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Turn on foreign keys in SQLite
cur.execute("PRAGMA foreign_keys = ON;")

# If you want to rerun safely, drop tables first (optional but recommended)
cur.executescript("""
DROP TABLE IF EXISTS price_metrics;
DROP TABLE IF EXISTS utilization;
DROP TABLE IF EXISTS drug;
""")
conn.commit()

# Create tables
cur.executescript("""
PRAGMA foreign_keys = ON;

-- 1) Drug identity table (one row per NDC)
CREATE TABLE drug (
    ndc TEXT PRIMARY KEY,
    labeler_code TEXT,
    product_code TEXT,
    package_size TEXT,
    product_name TEXT
);

-- 2) Raw utilization + reimbursement table (main fact table)
CREATE TABLE utilization (
    utilization_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ndc TEXT NOT NULL,
    utilization_type TEXT,
    state TEXT,
    year INTEGER,
    quarter INTEGER,
    suppression_used TEXT,
    units_reimbursed REAL,
    number_of_prescriptions INTEGER,
    total_amount_reimbursed REAL,
    medicaid_amount_reimbursed REAL,
    non_medicaid_amount_reimbursed REAL,
    FOREIGN KEY (ndc) REFERENCES drug(ndc)
);


-- Helpful indexes
CREATE INDEX IF NOT EXISTS idx_util_ndc ON utilization(ndc);
CREATE INDEX IF NOT EXISTS idx_util_state_time ON utilization(state, year, quarter);
CREATE INDEX IF NOT EXISTS idx_util_time ON utilization(year, quarter);

""")
conn.commit()

print("Tables are created")


Tables are created


In [16]:
# Select only drug identity columns
drug_df = df[[
    "ndc",
    "labeler_code",
    "product_code",
    "package_size",
    "product_name"
]].dropna(subset=["ndc"]).drop_duplicates(subset=["ndc"])

# Insert into drug table
cur.executemany(
    """
    INSERT OR REPLACE INTO drug
    (ndc, labeler_code, product_code, package_size, product_name)
    VALUES (?, ?, ?, ?, ?)
    """,
    drug_df.itertuples(index=False, name=None)
)

conn.commit()

# Check count
cur.execute("SELECT COUNT(*) FROM drug;")
print("Drug rows inserted:", cur.fetchone()[0])


Drug rows inserted: 46997


In [17]:
util_cols = [
    "ndc",
    "utilization_type",
    "state",
    "year",
    "quarter",
    "suppression_used",
    "units_reimbursed",
    "number_of_prescriptions",
    "total_amount_reimbursed",
    "medicaid_amount_reimbursed",
    "non_medicaid_amount_reimbursed"
]

util_df = df[util_cols].dropna(subset=["ndc", "state", "year", "quarter"])

cur.executemany(
    """
    INSERT INTO utilization (
        ndc,
        utilization_type,
        state,
        year,
        quarter,
        suppression_used,
        units_reimbursed,
        number_of_prescriptions,
        total_amount_reimbursed,
        medicaid_amount_reimbursed,
        non_medicaid_amount_reimbursed
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    util_df.itertuples(index=False, name=None)
)

conn.commit()

# Check count
cur.execute("SELECT COUNT(*) FROM utilization;")
print("Utilization rows inserted:", cur.fetchone()[0])


Utilization rows inserted: 2637009


In [18]:
#Verifying

print("Tables in database:")
print(pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
))

print("\nRow counts:")
for table in ["drug", "utilization"]:
    cur.execute(f"SELECT COUNT(*) FROM {table};")
    print(table, ":", cur.fetchone()[0])


Tables in database:
              name
0  sqlite_sequence
1             drug
2      utilization

Row counts:
drug : 46997
utilization : 2637009


In [19]:
query = """
SELECT u.state,
       u.year,
       u.quarter,
       d.product_name,
       u.number_of_prescriptions,
       u.total_amount_reimbursed
FROM utilization u
JOIN drug d ON u.ndc = d.ndc
LIMIT 10;
"""

pd.read_sql_query(query, conn)


,state,year,quarter,product_name,number_of_prescriptions,total_amount_reimbursed
0,AK,2025,2,TRULICITY,107.0,102976.40
1,AK,2025,2,TRULICITY,109.0,104481.92
2,AK,2025,2,EMGALITY P,20.0,15227.25
3,AK,2025,2,TALTZ AUTO,30.0,231532.28
4,AK,2025,2,TALTZ AUTO,NaN,NaN
5,AK,2025,2,MOUNJARO,104.0,108908.80
6,AK,2025,2,MOUNJARO,60.0,62499.89
7,AK,2025,2,MOUNJARO,85.0,89214.16
8,AK,2025,2,MOUNJARO,97.0,103266.58
9,AK,2025,2,MOUNJARO,89.0,93349.46


In [20]:
conn.close()
print("Database successfully built.")


Database successfully built.
